In [1]:
import re
import time
import string
# pip install nltk
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import numpy as np
# pip install matplotlib
import matplotlib.pyplot as plt
# pip install susi
import susi
# pip install -U scikit-learn
import sklearn.metrics
import pandas as pd
# pip install seaborn
import seaborn as sn



d = 100
H = {}
s = {}
s_test = {}
n = 3
np.random.seed(42)
language_codes = ['bg', 'cs',  'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'it', 'lt', 'lv', 'nl', 'pl', 'pt', 'ro', 'sk', 'sl', 'sv']
permutation = np.random.permutation(d)
training_path = 'data/training/'
test_path = 'data/test/'
X_train = np.empty((400000, d))
X_test = np.empty((400000, d))
Y_train = np.empty((400000, ), dtype=object)
Y_test = np.empty((400000, ), dtype=object)
H_n_gram = {}
# Function to generate n-grams from a sentence
def generate_ngrams(text):
    # Generate n-grams
    n_grams = list(ngrams(list(text), n))
    
    return n_grams
    
def bind(H1, H2):
    return np.multiply(H1, H2)

def permute(H, r):
    for _ in range(r):
        H = H[permutation]
    return H

def bundle(H1, H2):
    return H1 + H2
    
def generate_hd_vector_letter():
    return np.random.choice([-1, 1], size = d)

def generate_hd_vector_n_gram(n_gram):
    if n_gram in H_n_gram:
        return H_n_gram[n_gram]
        
    result = np.ones(d)
    for i in range(n):
        result = bind(permute(H[n_gram[i]], i + 1), result)
    H_n_gram[n_gram] = result
    return result
    
def generate_hd_vector_text(n_grams, scope):  
    n_grams = set(n_grams)
    result = np.zeros(d)
    for n_gram in n_grams:
        rep = s[n_gram] if scope == 'train' else s_test[n_gram]
        result += rep * generate_hd_vector_n_gram(n_gram)
    
    return np.interp(result, (result.min(), result.max()), (-1, 1))

def preprocess_text(sentence):
    # Remove punctuation using a regular expression
    expr = r'[^\w\s]'
    #expr = r'[^a-zA-Z\s]'
    #expr = r'[^a-zA-Z\sÀ-ÖØ-öø-ÿ]'
    sentence = re.sub(expr, '', sentence)
    # Convert to lowercase
    sentence = sentence.lower()
    return sentence
    
#def preprocess_text(sentence):
#    # Remove punctuation
#    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
#    # Convert to lowercase
#    sentence = sentence.lower()
#    return sentence
ind_train = 0
ind_test = 0
def load_training_data(file_path, ind):
    global X_train, Y_train, ind_train
    i = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            
            # Split the line into number and sentence
            parts = line.strip().split('\t', 1)
            if len(parts) == 2:
                _, text = parts
                text = preprocess_text(text)
                for letter in text:
                    if not letter in H:
                        H[letter] = generate_hd_vector_letter()  
                n_grams = generate_ngrams(text)
                for n_gram in n_grams:
                    if n_gram in s:
                        s[n_gram] += 1
                    else:
                        s[n_gram] = 1    
                #X_train = np.vstack([X_train, generate_hd_vector_text(n_grams, 'train')])
                #Y_train = np.append(Y_train, language_codes[ind])
                X_train[ind_train] = generate_hd_vector_text(n_grams, 'train')
                Y_train[ind_train] = language_codes[ind]    
                ind_train += 1
                if ind_train == 100000:
                    break
                        
def load_test_data(file_path, ind):
    global X_test, Y_test, ind_test
    i = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for text in file:
            if text[0] != '<':
                text = preprocess_text(text)
                for letter in text:
                    if not letter in H:
                        H[letter] = generate_hd_vector_letter()  
                n_grams = generate_ngrams(text)
                for n_gram in n_grams:
                    if n_gram in s_test:
                        s_test[n_gram] += 1
                    else:
                        s_test[n_gram] = 1    
                #X_test = np.vstack([X_test, generate_hd_vector_text(n_grams, 'test')])
                #Y_test = np.append(Y_test, language_codes[ind])
                X_test[ind_test] = generate_hd_vector_text(n_grams, 'test')
                Y_test[ind_test] = language_codes[ind]
                ind_test += 1
                if ind_test == 100000:
                    break
                

start_time = time.time()

for i in range(len(language_codes)):
    load_training_data(f"{training_path}/{language_codes[i]}.txt", i)
    load_test_data(f"{test_path}/{language_codes[i]}.txt", i)
    print(f"loading data for {language_codes[i]} done ({i+1}/{len(language_codes)})")
    
train_permutation = np.random.permutation(ind_train)
test_permutation = np.random.permutation(ind_test)

X_train = X_train[0:ind_train][train_permutation]
Y_train = Y_train[0:ind_train][train_permutation]
X_test = X_test[0:ind_test][test_permutation]
Y_test = Y_test[0:ind_test][test_permutation]

end_time = time.time()
print(f"loading data took {end_time - start_time} seconds")

# initialize and fit SuSi
som = susi.SOMClassifier()
# som.n_iter_supervised = 1000
# som.train_mode_supervised = "mean"
som.n_rows = 1
som.n_columns = 21
start_time = time.time()
som.fit(X_train, Y_train)
end_time = time.time()
print(f"training took {end_time - start_time} seconds")

# predict and calculate the accuracy score
y_pred = som.predict(X_test)
print(som.score(X_test, Y_test))

def pred(X_test, som):
    mini = 1
    res = 20
    
    yy = np.empty((0, 1))
    for x in X_test:
        i = 0
        for ww in som.unsuper_som_:
            for w in ww:
                coss = np.dot(x, w)/(np.linalg.norm(x)*np.linalg.norm(w))
                if coss < mini:
                    mini = coss
                    res = i
                i += 1
        yy = np.append(yy, res)
    return yy
    
# y = pred(X_test, som)
# dif = abs(Y_test-y)
# print(len([x for x in dif if int(x)==0])/len(dif))

conf_mat= sklearn.metrics.confusion_matrix(Y_test, y_pred, labels=None, sample_weight=None)

#plot confusion matrix
%matplotlib inline
df_cm = pd.DataFrame(conf_mat, index = [i for i in language_codes], columns = [j for j in language_codes])
plt.figure(figsize = (18,18))
sn.heatmap(df_cm, annot=True)
        
                

    

loading data took 0.13060331344604492 seconds


ValueError: Found array with 0 sample(s) (shape=(0, 100)) while a minimum of 1 is required.

In [297]:
np.save('X_train.npy', X_train)
np.save('Y_train.npy', Y_train)
np.save('X_test.npy', X_test)
np.save('Y_test.npy', Y_test)
# Load training and test data
X_train = np.load('X_train.npy', allow_pickle = True)
Y_train = np.load('Y_train.npy', allow_pickle = True)
X_test = np.load('X_test.npy', allow_pickle = True)
Y_test = np.load('Y_test.npy', allow_pickle = True)
print(Y_train)

['da' 'nl' 'en' ... 'nl' 'fr' 'lt']
